In [1]:
class Segment(object):
    """
    A class representing a single segment. Each segment has a left
    and right, denoting the loci over which it spans, a node and a
    next, giving the next in the chain.
    """
    def __init__(self, index):
        self.left = None
        self.right = None
        self.node = None
        self.prev = None
        self.next = None
        self.population = None
        self.index = index

    def __str__(self):
        s = "({0}:{1}-{2}->{3}: prev={4} next={5})".format(
            self.index, self.left, self.right, self.node, repr(self.prev),
            repr(self.next))
        print("reprt is %s" %s)
        return s
    #Q# what is this for??
    def __lt__(self, other):
        return ((self.left, self.right, self.population, self.node)
                < (other.left, other.right, other.population, self.node))

In [ ]:
s = Segment(0)

s.left=0 ;s.right = 3; s.node = 1; s.population = 0 ;s.prev = None; s.next = 0

s1 = Segment(0)
s1.left=5 ;s1.right = 10; s1.node = 1; s1.population = 0 ;s1.prev = None; s1.next = 1
# s1= Segment(2)
s.__str__()
s1.__str__()
s.__lt__(s1)

# Fenwick algorithm  or  Binary Indexed Tree

Great explanation  found at: 
https://www.geeksforgeeks.org/binary-indexed-tree-or-fenwick-tree-2/

Youtube video:
https://www.youtube.com/watch?v=v_wj_mOAlig


In [41]:
# my implementation of Binary indexed tree: 

def sum_BIT(BITree, index):
    
    '''
    Returns sum of arr[0..index]. This function assumes 
        that the array is preprocessed and partial sums of 
        array elements are stored in BITree[].
    '''
    #index in BIT is 1 more than array , we dont care about index 0 in BITree    
    index +=1
    s=0
    #Traverse ancestors of BITree[index]
    while (index>0):
        s += BITree[index]
        index -= (index & (-index))#  remove the last set bit, "up and left" in explanation.
    return s

def ADD(BITree,n, index, val):
    '''
    add val to index and update the BITree , n is the length of the array
    '''
    # index in BITree, we dont care about index 0 in BITree
    index += 1 
    # Traverse all ancestors and add 'val'
    while (index <= n):
        BITree[index] += val
        index += (index & (-index))# add the last set bit, " right and up" in explanation.
    return BITree

def constructB(arr,n):
    '''n is the length of arr'''
    # Create and initialize BITree[] as 0
    BITree = [0]*(n+1)  
    for i in range(n):
        ADD(BITree = BITree,n = n, index = i, val = arr[i])
    return BITree
        
# test 
arr =  [2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9]
BITree = constructB(arr, len(arr))

print("Sum of elements in arr[0..5] is " + str(sum_BIT(BITree, 5))) 
BITree1 = ADD(BITree, len(arr), 5, 6)# add value 6 to index 5
print("Sum of elements in arr[0..5] is " + str(sum_BIT(BITree1, 5))) 

        

Sum of elements in arr[0..5] is 12
Sum of elements in arr[0..5] is 6


## The following is implemented by Kelleher

In [17]:

class FenwickTree(object):
    """
    A Fenwick Tree to represent cumulative frequency tables over
    integers. Each index from 1 to max_index initially has a
    zero frequency.
    This is an implementation of the Fenwick tree (also known as a Binary
    Indexed Tree) based on "A new data structure for cumulative frequency
    tables", Software Practice and Experience, Vol 24, No 3, pp 327 336 Mar
    1994. This implementation supports any non-negative frequencies, and the
    search procedure always returns the smallest index such that its cumulative
    frequency <= f. This search procedure is a slightly modified version of
    that presented in Tech Report 110, "A new data structure for cumulative
    frequency tables: an improved frequency-to-symbol algorithm." available at
    https://www.cs.auckland.ac.nz/~peter-f/FTPfiles/TechRep110.ps
    """
    def __init__(self, max_index):
        assert max_index > 0
        self.__max_index = max_index
        self.__tree = [0 for j in range(max_index + 1)]## all zero, the index is one more than the arr
        # Compute the binary logarithm of max_index
        u = self.__max_index
        while u != 0:
            self.__log_max_index = u
            u -= (u & -u)
            
    def get_cumulative_frequency(self, index): ## this is equivalent as sumBIT()
        """
        Returns the cumulative frequency of the specified index.
        """
        assert 0 < index <= self.__max_index
        j = index
        s = 0
        while j > 0:
            s += self.__tree[j]
            j -= (j & -j)## remove the last set bit
        return s
    def get_total(self):
        """
        Returns the total cumulative frequency over all indexes.
        """
        return self.get_cumulative_frequency(self.__max_index)

    def increment(self, index, v):## equivalent to my ADD()
        """
        Increments the frequency of the specified index by the specified
        value.
        """
        assert 0 < index <= self.__max_index
        j = index
        while j <= self.__max_index:
            self.__tree[j] += v
            j += (j & -j)  ## add the last set bit 

    def set_value(self, index, v):
        """
        Sets the frequency at the specified index to the specified value.
        """
        f = self.get_frequency(index)
        self.increment(index, v - f)



    def get_frequency(self, index):
        """
        Returns the frequency of the specified index.
        """
        assert 0 < index <= self.__max_index
        j = index
        v = self.__tree[j]
        p = j & (j - 1)# 
        j -= 1
        while p != j:
            v -= self.__tree[j]
            j = j & (j - 1)## amazing, exactly gives you the number of links  of segment with idex =index
        return v

    def find(self, v):
        """
        Returns the smallest index with cumulative sum >= v.
        """
        j = 0
        s = v
        half = self.__log_max_index
        while half > 0:
            # Skip non-existant entries
            while j + half > self.__max_index:
                half >>= 1
            k = j + half
            if s > self.__tree[k]:
                j = k
                s -= self.__tree[j]
            half >>= 1
        return j + 1



In [36]:
def twos_complement(input_value, num_bits):
	'''Calculates a two's complement integer from the given input value's bits'''
	mask = 2**(num_bits - 1)
	return -(input_value & mask) + (input_value & ~mask)

In [40]:
twos_complement(9, 8)

9